In [2]:
import numpy as np
import pandas as pd

from ComputeNode import ComputeNode

In [49]:
import xarray as xr

# MDHD Model

We'll start by recreating all the model steps here in this notebook, before re-writing them with the ComputeNode package.

## Inputs

### Directory Information

In [21]:
# File Directories
IMPORT_FOLDER = "../Data/Inputs/"
ECON_FOLDER = "Econ/"
DEMAND_SCENARIOS_FOLDER = "Demand_Scenarios/"
VEHICLE_STOCK_FOLDER = "Vehicle_Stock_Attributes/"
VMT_ENERGY_DEMAND_FOLDER = "VMT_Energy_Demand/"
REGIONAL_MODULE_FOLDER = "Regional_Module/"
OTHER_FOLDER = "Other/" # TODO: what is that?

# New Directories
INDICES_FOLDER = "../Data/Indices/"

### File Names

In [22]:
# Economic File Names
GSP_FILE_NAME = "Economic_Activity_Indicator_202307.xlsx"
GSP_SHEET_NAME = "Regional Mid"
FUEL_PRICE_FILE_NAME = "Fuel price forecast 2023-09-22.xlsx"
FAF_FILE_NAME = "Linear_interpolation_output_06.22.22-1623 named.xlsx"

In [23]:
# Vehicle Stock and Attributes File Names
MAINTENANCE_COST_FILE_NAME = "maintenance costs v24 2022 import.xlsx"
TRUCK_PRICE_FILE_NAME = "2023 IEPR Truck Prices - C.xlsx"
FUEL_PER_TRUCK_MI_FILE_NAME = "FFUPTM_20210603.xlsx"
LOAD_FRACTION_FILE_NAME = "t21 truck loaded fraction 20210618 RMc.xlsx"
SURVIVAL_RATE_FILE_NAME = "EMFAC_Survival_Rates_.06.04.21-1411 20221129 modtxt RMc.xlsx"
ACT_ACF_ASSIGNED_FILE_NAME = "ARB estimate ACT ACF fraction of stock 20220805 RMc.xlsx"
CLASS_BY_FLEET_SIZE_TYPE_OWNER_FILE_NAME = "MDHD_Stock_CY19_20220915 plus.xlsx"

In [78]:
# Indices File Names
COUNTIES_FILE_NAME = "County_names.xlsx"
ZONES_FILE_NAME = "CA_zones.xlsx"

### Indices, Counters, and Other Final Variables

The idea here is to standardize conventions across our model so we don't run into naming errors.

In [82]:
# INDICES

# YEARS
YEARS = range(2019, 2051)

# list of counties
COUNTIES = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne',
       'Ventura', 'Yolo', 'Yuba']

# FAF Zones
ZONES = ['CA Los A',
 'CA rem',
 'CA Sacra',
 'CA San D',
 'CA San J',
 'CA Fresno',
 'OOS_I-10_201to500',
 'OOS_I-10_over500',
 'OOS_I-15_201to500',
 'OOS_I-15_over500',
 'OOS_I-40_201to500',
 'OOS_I-40_over500',
 'OOS_I-5_201to500',
 'OOS_I-5_over500',
 'OOS_I-8_201to500',
 'OOS_I-8_over500',
 'OOS_I-8_to200',
 'OOS_I-80_201to500',
 'OOS_I-80_over500',
 'OOS_I-80_to200']

CA_ZONES = ['CA rem', 'CA San J', 'CA Sacra', 'CA Los A', 'CA Fresno', 'CA San D']

In [91]:
# FINAL VARIABLES

COUNTY_LABEL = 'County'
ZONE_LABEL = 'Zone'
YEAR_LABEL = 'Year'

### Data Cleaning

In [92]:
gsp_data = pd.read_excel(IMPORT_FOLDER + ECON_FOLDER + GSP_FILE_NAME, sheet_name=GSP_SHEET_NAME)
gsp_data.head()

,County,Zone,2019,2020,2021,2022,2023,2024,2025,2026,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Alameda,CA San J,160.404767,157.339722,168.745442,166.408824,170.307781,173.731745,178.683926,184.232453,...,259.812856,265.229528,270.733966,276.347346,282.091207,287.881132,293.824507,300.183569,306.741765,313.442232
1,Alpine,CA rem,0.235178,0.207040,0.219193,0.224208,0.227723,0.227719,0.230208,0.233819,...,0.290995,0.295237,0.299536,0.303867,0.308217,0.312477,0.316835,0.321617,0.326499,0.331166
2,Amador,CA rem,2.095760,2.073996,2.099969,2.100453,2.160152,2.163390,2.199133,2.240731,...,2.770305,2.808531,2.848202,2.889100,2.930612,2.971606,3.013864,3.060628,3.108769,3.156694
3,Butte,CA rem,14.061553,13.291727,13.615938,13.218811,13.489031,13.729690,14.080265,14.481030,...,19.631352,19.989597,20.354803,20.725771,21.102527,21.479162,21.867346,22.286651,22.720523,23.161941
4,Calaveras,CA rem,1.591438,1.613576,1.668310,1.694217,1.739392,1.744850,1.769649,1.799966,...,2.135361,2.158059,2.181630,2.205694,2.229704,2.253127,2.277811,2.305962,2.335095,2.363785


In [93]:
gsp_data_xr = xr.DataArray(gsp_data.iloc[:, 2:], dims=[COUNTY_LABEL, YEAR_LABEL], coords={COUNTY_LABEL: COUNTIES, YEAR_LABEL: YEARS}) # Convert data into XArray
gsp_data_xr = gsp_data_xr.assign_coords({ZONE_LABEL: (COUNTY_LABEL, gsp_data['Zone'].to_list())}) # add additional coordinates for FAF zone
gsp_data_xr

<xarray.DataArray (County: 58, Year: 32)>
array([[1.60404767e+02, 1.57339722e+02, 1.68745442e+02, ...,
        3.00183569e+02, 3.06741765e+02, 3.13442232e+02],
       [2.35178043e-01, 2.07039608e-01, 2.19192628e-01, ...,
        3.21617449e-01, 3.26498567e-01, 3.31166353e-01],
       [2.09575982e+00, 2.07399592e+00, 2.09996906e+00, ...,
        3.06062759e+00, 3.10876873e+00, 3.15669401e+00],
       ...,
       [6.23381008e+01, 6.24288664e+01, 6.47228313e+01, ...,
        1.06840453e+02, 1.09057153e+02, 1.11321905e+02],
       [1.92782012e+01, 1.90344216e+01, 1.95994717e+01, ...,
        2.95100774e+01, 2.99662573e+01, 3.04282224e+01],
       [2.98352863e+00, 3.13524947e+00, 3.18620649e+00, ...,
        5.67588478e+00, 5.80262439e+00, 5.93278060e+00]])
Coordinates:
  * County   (County) <U15 'Alameda' 'Alpine' 'Amador' ... 'Yolo' 'Yuba'
  * Year     (Year) int32 2019 2020 2021 2022 2023 ... 2046 2047 2048 2049 2050
    Zone     (County) <U9 'CA San J' 'CA rem' 'CA rem' ... 'CA Sacra' 'CA Sacra'

In [102]:
gsp_data_xr.where(gsp_data_xr.Zone =='CA San J', drop=True).to_pandas()

Year,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
County,,,,,,,,,,,,,,,,,,,,,
Alameda,160.404767,157.339722,168.745442,166.408824,170.307781,173.731745,178.683926,184.232453,189.567747,194.802472,...,259.812856,265.229528,270.733966,276.347346,282.091207,287.881132,293.824507,300.183569,306.741765,313.442232
Contra Costa,87.344779,74.842965,77.920437,76.718545,78.181684,80.171487,82.631421,85.415354,88.124554,90.824632,...,121.599742,124.154194,126.773817,129.453539,132.163308,134.881039,137.716078,140.781014,143.968355,147.242122
Marin,21.877966,21.557364,22.270359,21.696367,22.050849,22.554758,23.248040,24.026960,24.791345,25.548196,...,34.623959,35.345797,36.080659,36.834375,37.606021,38.384881,39.192537,40.062527,40.962830,41.885546
Napa,12.915806,12.178606,12.696009,12.491741,12.770083,13.014220,13.362807,13.757470,14.134848,14.504795,...,18.820006,19.174897,19.536884,19.905362,20.280465,20.655864,21.042915,21.460376,21.892064,22.332050
San Benito,3.631156,3.719417,3.782819,3.594829,3.801045,3.948241,4.139077,4.342847,4.541040,4.730973,...,7.208836,7.428916,7.655820,7.889395,8.128821,8.372238,8.628475,8.901283,9.185314,9.478151
San Francisco,189.616314,189.632265,205.346947,210.002489,216.795333,221.534582,227.071270,233.114628,238.446734,243.917397,...,321.179908,328.126843,335.284718,342.695371,350.353377,358.150811,366.265597,374.972452,383.979799,393.233024
San Joaquin,39.193460,39.344947,41.068251,41.083636,41.995320,42.769375,43.873524,45.127384,46.320706,47.484063,...,60.634542,61.684866,62.748592,63.823261,64.909421,65.988381,67.095092,68.289451,69.524431,70.778931
San Mateo,118.523297,120.560262,138.575004,144.326575,149.821154,153.988778,159.256031,164.908682,169.968437,174.707894,...,230.629408,235.560257,240.635702,245.881944,251.314777,256.844963,262.561742,268.657089,274.952818,281.460436
Santa Clara,323.214651,334.432123,369.171473,363.445103,371.112564,378.411004,388.596701,399.952147,410.759976,421.444935,...,560.634201,573.021868,585.776932,598.902326,612.429542,626.115751,640.276675,655.471293,671.241301,687.407307


In [76]:
gsp_data_xr.Zones

<xarray.DataArray 'Zones' (County: 58)>
array(['CA San J', 'CA rem', 'CA rem', 'CA rem', 'CA rem', 'CA rem',
       'CA San J', 'CA rem', 'CA Sacra', 'CA Fresno', 'CA rem', 'CA rem',
       'CA rem', 'CA rem', 'CA rem', 'CA rem', 'CA rem', 'CA rem',
       'CA Los A', 'CA Fresno', 'CA San J', 'CA rem', 'CA rem', 'CA rem',
       'CA rem', 'CA rem', 'CA rem', 'CA San J', 'CA Sacra', 'CA Los A',
       'CA Sacra', 'CA rem', 'CA Los A', 'CA Sacra', 'CA San J',
       'CA Los A', 'CA San D', 'CA San J', 'CA San J', 'CA rem',
       'CA San J', 'CA rem', 'CA San J', 'CA San J', 'CA rem', 'CA rem',
       'CA rem', 'CA San J', 'CA San J', 'CA rem', 'CA Sacra', 'CA rem',
       'CA rem', 'CA rem', 'CA rem', 'CA Los A', 'CA Sacra', 'CA Sacra'],
      dtype='<U9')
Coordinates:
  * County   (County) <U15 'Alameda' 'Alpine' 'Amador' ... 'Yolo' 'Yuba'
    Zones    (County) <U9 'CA San J' 'CA rem' 'CA rem' ... 'CA Sacra' 'CA Sacra'

In [75]:
ca_trans_gsp = gsp_data_xr.where(gsp_data_xr.Zones.isin(CA_ZONES))
ca_trans_gsp.Zones

<xarray.DataArray 'Zones' (County: 58)>
array(['CA San J', 'CA rem', 'CA rem', 'CA rem', 'CA rem', 'CA rem',
       'CA San J', 'CA rem', 'CA Sacra', 'CA Fresno', 'CA rem', 'CA rem',
       'CA rem', 'CA rem', 'CA rem', 'CA rem', 'CA rem', 'CA rem',
       'CA Los A', 'CA Fresno', 'CA San J', 'CA rem', 'CA rem', 'CA rem',
       'CA rem', 'CA rem', 'CA rem', 'CA San J', 'CA Sacra', 'CA Los A',
       'CA Sacra', 'CA rem', 'CA Los A', 'CA Sacra', 'CA San J',
       'CA Los A', 'CA San D', 'CA San J', 'CA San J', 'CA rem',
       'CA San J', 'CA rem', 'CA San J', 'CA San J', 'CA rem', 'CA rem',
       'CA rem', 'CA San J', 'CA San J', 'CA rem', 'CA Sacra', 'CA rem',
       'CA rem', 'CA rem', 'CA rem', 'CA Los A', 'CA Sacra', 'CA Sacra'],
      dtype='<U9')
Coordinates:
  * County   (County) <U15 'Alameda' 'Alpine' 'Amador' ... 'Yolo' 'Yuba'
    Zones    (County) <U9 'CA San J' 'CA rem' 'CA rem' ... 'CA Sacra' 'CA Sacra'

In [51]:
gsp_data.shape

(58, 34)

## Regional Forecast

In [45]:
ca_trans_gsp = gsp_data[gsp_data['Zone'].isin(CA_ZONES)]
ca_trans_gsp.head()

,County,Zone,2019,2020,2021,2022,2023,2024,2025,2026,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Alameda,CA San J,160.404767,157.339722,168.745442,166.408824,170.307781,173.731745,178.683926,184.232453,...,259.812856,265.229528,270.733966,276.347346,282.091207,287.881132,293.824507,300.183569,306.741765,313.442232
1,Alpine,CA rem,0.235178,0.207040,0.219193,0.224208,0.227723,0.227719,0.230208,0.233819,...,0.290995,0.295237,0.299536,0.303867,0.308217,0.312477,0.316835,0.321617,0.326499,0.331166
2,Amador,CA rem,2.095760,2.073996,2.099969,2.100453,2.160152,2.163390,2.199133,2.240731,...,2.770305,2.808531,2.848202,2.889100,2.930612,2.971606,3.013864,3.060628,3.108769,3.156694
3,Butte,CA rem,14.061553,13.291727,13.615938,13.218811,13.489031,13.729690,14.080265,14.481030,...,19.631352,19.989597,20.354803,20.725771,21.102527,21.479162,21.867346,22.286651,22.720523,23.161941
4,Calaveras,CA rem,1.591438,1.613576,1.668310,1.694217,1.739392,1.744850,1.769649,1.799966,...,2.135361,2.158059,2.181630,2.205694,2.229704,2.253127,2.277811,2.305962,2.335095,2.363785


In [47]:
ca_trans_gsp.set_index('Zone').groupby('County').sum()

,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
County,,,,,,,,,,,,,,,,,,,,,
Alameda,160.404767,157.339722,168.745442,166.408824,170.307781,173.731745,178.683926,184.232453,189.567747,194.802472,...,259.812856,265.229528,270.733966,276.347346,282.091207,287.881132,293.824507,300.183569,306.741765,313.442232
Alpine,0.235178,0.207040,0.219193,0.224208,0.227723,0.227719,0.230208,0.233819,0.237714,0.241623,...,0.290995,0.295237,0.299536,0.303867,0.308217,0.312477,0.316835,0.321617,0.326499,0.331166
Amador,2.095760,2.073996,2.099969,2.100453,2.160152,2.163390,2.199133,2.240731,2.279528,2.320873,...,2.770305,2.808531,2.848202,2.889100,2.930612,2.971606,3.013864,3.060628,3.108769,3.156694
Butte,14.061553,13.291727,13.615938,13.218811,13.489031,13.729690,14.080265,14.481030,14.863706,15.239358,...,19.631352,19.989597,20.354803,20.725771,21.102527,21.479162,21.867346,22.286651,22.720523,23.161941
Calaveras,1.591438,1.613576,1.668310,1.694217,1.739392,1.744850,1.769649,1.799966,1.827854,1.856340,...,2.135361,2.158059,2.181630,2.205694,2.229704,2.253127,2.277811,2.305962,2.335095,2.363785
Colusa,2.183227,2.033245,2.000256,1.999437,2.111419,2.121666,2.166101,2.213097,2.256585,2.296997,...,2.698486,2.731914,2.766705,2.802520,2.839844,2.876514,2.913963,2.955371,2.997938,3.040529
Contra Costa,87.344779,74.842965,77.920437,76.718545,78.181684,80.171487,82.631421,85.415354,88.124554,90.824632,...,121.599742,124.154194,126.773817,129.453539,132.163308,134.881039,137.716078,140.781014,143.968355,147.242122
Del Norte,1.215711,1.221405,1.222967,1.236063,1.271043,1.271154,1.283860,1.297617,1.308805,1.321232,...,1.467374,1.480187,1.493763,1.507932,1.522269,1.536110,1.550377,1.566760,1.583622,1.600042
El Dorado,9.734230,9.268060,9.877608,9.747607,9.988796,10.162622,10.427917,10.740226,11.038501,11.323305,...,14.573743,14.837792,15.106574,15.377896,15.650958,15.921015,16.200091,16.502202,16.815992,17.134908


In [39]:
assert np.all(gsp_data['County'] == COUNTIES)

In [7]:
pd.read_excel("./poopoo.xlsx")

,baseline,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,dimension,FuelPricePerUnit:time:year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fuelType,2019,2020,2021,2022,2023,2024,2025,2026,2027,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
2,fuel,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,...,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon,dollars/gallon
3,electric,6.316831,6.554084,6.380168,6.547692,7.568715,7.720199,7.59367,7.678162,7.818728,...,8.227332,8.268392,8.315781,8.371021,8.431067,8.473223,8.515589,8.558167,8.600958,8.643962
4,hydrogen,18.82482,18.472455,16.952587,17.161667,27.993333,20,19,18,17,...,9.82,9.82,9.82,9.82,9.82,9.82,9.82,9.82,9.82,9.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Diesel-Electric Hybrid,4.444268,3.774928,4.454678,6.028,4.686295,4.691048,4.69311,4.714986,4.729786,...,4.947314,4.955186,4.95941,4.961491,4.960135,4.955857,4.948507,4.937806,4.925728,4.916799
98,gasoline,4.158543,3.499508,4.385614,5.406,4.221592,4.222155,4.287246,4.294513,4.308734,...,4.633302,4.636543,4.63627,4.632327,4.626182,4.616672,4.605341,4.592318,4.576972,4.563406
99,hybrid,4.158543,3.499508,4.385614,5.406,4.221592,4.222155,4.287246,4.294513,4.308734,...,4.633302,4.636543,4.63627,4.632327,4.626182,4.616672,4.605341,4.592318,4.576972,4.563406
100,natural gas,3.515841,3.089174,2.90213,2.843154,2.764874,3.080105,3.146741,3.218041,3.190041,...,3.889145,3.987877,4.110633,4.286754,4.33391,4.222946,4.456917,4.679933,4.790509,4.871859
